In [2]:
import numpy as np
import joblib
import cv2
import glob

# Target image size to resize all inputs to a consistent shape
IMG_SIZE = (64, 64)

# Load the best-performing model along with associated preprocessing tools
model = joblib.load('models/image_model.pkl')              # Best model for prediction
le = joblib.load('models/label_encoder.pkl')               # Label encoder for class name decoding
scaler = joblib.load('models/scaler.pkl')                  # Feature scaler used during training

# Load individual models for comparison (optional)
model_lr = joblib.load('models/logistic_regression_model.pkl')   # Logistic Regression
model_knn = joblib.load('models/knn_model.pkl')                  # K-Nearest Neighbors
model_perc = joblib.load('models/perceptron_model.pkl')          # Perceptron

def preprocess_image(path):
    """
    Reads and preprocesses a single image for model prediction.

    Steps:
        - Reads image using OpenCV
        - Resizes to fixed size
        - Normalizes pixel values
        - Scales features using training scaler

    Returns:
        Scaled feature vector if valid, else None
    """
    img = cv2.imread(path)
    if img is None:
        print(f"Warning: Unable to read image {path}")
        return None
    img = cv2.resize(img, IMG_SIZE)
    img = img.flatten() / 255.0
    img = scaler.transform([img])
    return img

print("Predicting other images:")

# Iterate over all images in the 'images/' folder
for img_path in glob.glob('images/*'):
    img_proc = preprocess_image(img_path)
    if img_proc is None:
        continue  # Skip unreadable images

    # Prediction using the best model
    pred_best = model.predict(img_proc)
    pred_best_label = le.inverse_transform(pred_best)[0]

    # Predictions using all models (for comparison/reporting)
    pred_lr = model_lr.predict(img_proc)
    pred_knn = model_knn.predict(img_proc)
    pred_perc = model_perc.predict(img_proc)

    # Output predictions from each model
    print(f"Image: {img_path}")
    print(f" Best Model Prediction: {pred_best_label}")
    print(f" Logistic Regression Prediction: {le.inverse_transform(pred_lr)[0]}")
    print(f" KNN Prediction: {le.inverse_transform(pred_knn)[0]}")
    print(f" Perceptron Prediction: {le.inverse_transform(pred_perc)[0]}")
    print("-----------")


Predicting other images:
Image: images\cat1.jpeg
 Best Model Prediction: dog
 Logistic Regression Prediction: dog
 KNN Prediction: cat
 Perceptron Prediction: cat
-----------
Image: images\cat2.jpg
 Best Model Prediction: cat
 Logistic Regression Prediction: cat
 KNN Prediction: dog
 Perceptron Prediction: cat
-----------
Image: images\cat3.jpeg
 Best Model Prediction: cat
 Logistic Regression Prediction: cat
 KNN Prediction: cat
 Perceptron Prediction: cat
-----------
Image: images\dog1.jpg
 Best Model Prediction: dog
 Logistic Regression Prediction: dog
 KNN Prediction: dog
 Perceptron Prediction: dog
-----------
Image: images\dog2.jpeg
 Best Model Prediction: dog
 Logistic Regression Prediction: dog
 KNN Prediction: cat
 Perceptron Prediction: dog
-----------
Image: images\dog3.jpg
 Best Model Prediction: dog
 Logistic Regression Prediction: dog
 KNN Prediction: dog
 Perceptron Prediction: dog
-----------


### Methods

* Implemented Logistic Regression, K-Nearest Neighbors (KNN), and Perceptron classifiers.
* Preprocessed images by resizing to 64×64 pixels, flattening, normalizing, and scaling.
* Trained on 2000 labeled images and tested on 10 test images.
* Used tuned parameters: Logistic Regression (`max_iter=1000`), KNN (`n_neighbors=5`), Perceptron (`max_iter=1000`, `tol=1e-3`).

---

### Results

| Model               | Test Accuracy |
| ------------------- | ------------- |
| Logistic Regression | 60%           |
| KNN                 | 50%           |
| Perceptron          | 50%           |

* Logistic Regression had the highest accuracy on the test set and was saved as the best model.

* Tested on 7 internet images:

  * The best model (logistic regression) correctly predicted some images but misclassified others (e.g., predicted some cats as dogs).
  * KNN and Perceptron predictions varied, sometimes correctly classifying images where logistic regression failed.

---

### Analysis

* Logistic Regression performed best overall but was not fully reliable on unseen internet images.
* KNN and Perceptron showed inconsistent but occasionally complementary predictions.

---

### Conclusion

The Logistic Regression model achieved the best test accuracy but struggled with some real-world internet images. While reasonably effective, the model is prone to errors, suggesting the need for more robust features or advanced methods for improved generalization.